In [13]:
from pyspark.sql import SparkSession

# (8 cores, 16gb per machine) x 5 = 40 cores

# New API
spark_session = SparkSession\
        .builder\
        .master("local") \
        .appName("A.2")\
        .getOrCreate()

# Old API (RDD)
spark_context = spark_session.sparkContext

distFileEN = spark_context.textFile("europarl-v7.sv-en.en")
distFileSV = spark_context.textFile("europarl-v7.sv-en.sv")
TokenEN = distFileEN.pipe("/home/ubuntu/tools/tokenizer.perl -l en")
TokenSV = distFileSV.pipe("/home/ubuntu/tools/tokenizer.perl -l sv")
LowerCaseEN = TokenEN.flatMap(lambda line: line.lower().split("/n")).flatMap(lambda lst: ''.join(lst).split(" "))
LowerCaseSV = TokenSV.flatMap(lambda line: line.lower().split("/n")).flatMap(lambda lst: ''.join(lst).split(" "))

In [14]:
LowerCaseEN.take(5)
#['resumption', 'of', 'the', 'session', 'i']
LowerCaseSV.take(5)
#['återupptagande', 'av', 'sessionen', 'jag', 'förklarar']

['återupptagande', 'av', 'sessionen', 'jag', 'förklarar']

In [15]:
EN_mapped = LowerCaseEN.map(lambda x: (x,1))
EN_grouped = EN_mapped.groupByKey()
SV_mapped = LowerCaseSV.map(lambda x: (x,1))
SV_grouped = SV_mapped.groupByKey()

In [16]:
EN_words_freq = EN_grouped.mapValues(sum).map(lambda x: (x[1],x[0])).sortByKey(False)
SV_words_freq = SV_grouped.mapValues(sum).map(lambda x: (x[1],x[0])).sortByKey(False)

In [5]:
EN_words_freq.take(10)

[(3505228, 'the'),
 (2440596, ','),
 (1798163, '.'),
 (1662136, 'of'),
 (1543797, 'to'),
 (1319061, 'and'),
 (1089015, 'in'),
 (839170, 'that'),
 (774986, 'a'),
 (774949, 'is')]

In [17]:
SV_words_freq.take(10)

[(1849014, '.'),
 (1709915, 'att'),
 (1477733, ','),
 (1350825, 'och'),
 (1054362, 'i'),
 (952972, 'det'),
 (917581, 'som'),
 (914944, 'för'),
 (740761, 'av'),
 (701844, 'är')]